## `Mount Google Drive`

In [90]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [91]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#/dev/null:to avoid verbose

# Download and extract Hadoop
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz
!tar -xzf hadoop-3.4.0.tar.gz

## `Unzip the hadoop 3.4.0 to the directory`


In [ ]:
# Extract Hadoop to the directory
#!tar -xzf hadoop-3.4.0.tar.gz -C /content/drive/MyDrive/hadoopcolab
#if you want to change the unzipped file to a different directory, very slow

### Set Hadoop Environment Variables

In [ ]:
import os

# Set the path to your Hadoop installation
hadoop_path = '/content/hadoop-3.4.0'

# Set environment variables
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['HADOOP_HOME'] = hadoop_path
#os.environ['PATH'] = os.environ['PATH'] + f':{hadoop_path}/bin:{hadoop_path}/sbin'


## Create a directory in hadoop

In [ ]:
!$hadoop_path/bin/hadoop fs -mkdir -p /content/drive/MyDrive/My_Data


In [ ]:
!$hadoop_path/bin/hadoop fs -ls /content/drive/MyDrive/My_Data


In [ ]:
hd_path = '/content/hadoop-3.4.0/bin/hadoop'
dir_path = '/content/drive/MyDrive/My_Data'

In [ ]:
!$hd_path fs  -touchz $dir_path/file.txt

In [ ]:
%cd /content/drive/MyDrive/My_Data

/content/drive/MyDrive/My_Data


In [ ]:
!ls

file.txt


In [ ]:
!$hd_path fs -cat /content/drive/MyDrive/My_Data/file.txt

In [ ]:
%cd /content

/content


In [ ]:
new_data="house, dog, cat, rat, house, bee"
# Create a temporary file and append new data
temp_file="temp_file.txt"
!echo "$new_data" >> "$temp_file"
# Remove existing file and upload the updated file
!$hd_path fs -rm "$dir_path/file.txt"
!$hd_path fs -put -f "$temp_file" "$dir_path/file.txt"
# Clean up temporary file
!rm "$temp_file"
##will throw me conf infor

2024-05-22 19:37:41,697 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
Deleted /content/drive/MyDrive/My_Data/file.txt


In [ ]:
!$hd_path fs -cat "$dir_path/file.txt"

house, dog, cat, rat, house, bee


In [ ]:
hdfs_path = '/content/hadoop-3.4.0/bin/'
!$hdfs_path/hdfs dfs -ls

Found 2 items
-rw-------   1 root root         33 2024-05-22 19:37 file.txt
-rw-------   1 root root  965537117 2024-03-04 09:36 hadoop-3.4.0.tar.gz


In [ ]:
!$hdfs_path/hdfs dfs -ls

Found 2 items
-rw-------   1 root root         33 2024-05-22 19:37 file.txt
-rw-------   1 root root  965537117 2024-03-04 09:36 hadoop-3.4.0.tar.gz


In [ ]:
!pip install hdfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 743.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34324 sha256=1db39fc8ae63db17421ac576491baf16ce4160a98a5ffb349bab696d427f554d
  Stored in directory: /root/.cache/pip/wheels/e5/8d/b6/99c1c0a3ac5788c866b0ecd3f48b0134a5910e6ed26011800b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6fe0fe034be03c0ba210daeec9febe46a862860dcd1b8c960d363509396a4df0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built hdfs docopt


In [ ]:
# Create the 'sample_data' directory in HDFS
! {hd_path} fs -mkdir /mydata

## `Convert word file to txt`

In [ ]:
!pip install python-docx

In [ ]:
import docx
local_file_path = '/content/drive/MyDrive/HW6_Kowarsch.docx'

doc = docx.Document(local_file_path)
text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])

with open('/content/drive/MyDrive/HW6_Kowarsch.txt', 'w') as f:
    f.write(text)

In [ ]:
local_txt_file = '/content/drive/MyDrive/HW6_Kowarsch.txt'
hdfs_txt_file = '/mydata/HW6_Kowarsch.txt'
! {hd_path} fs -put -f $local_txt_file $hdfs_txt_file

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0d0d9b59c902d5f82b4c0b8ea9e2ea238464c2e20510575fd455472c44701038
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Clear the Spark session
spark.stop()
#reconnect the session
spark = SparkSession.builder.getOrCreate()

# Read the text file from HDFS
hdfs_txt_file = '/mydata/HW6_Kowarsch.txt' #replace this file with your own txt.file
df = spark.read.text(hdfs_txt_file)

# Perform word count
word_count = (df
              .select("value")
              .withColumn("words", f.split("value", " "))
              .select("words")
              .withColumn("word", f.explode("words"))
              .groupBy("word")
              .count()
              .orderBy("count", ascending=False))

# Display the results
word_count.show(n=20, truncate=False)

+-------+-----+
|word   |count|
+-------+-----+
|the    |76   |
|       |71   |
|of     |47   |
|to     |30   |
|is     |26   |
|a      |26   |
|in     |20   |
|The    |16   |
|that   |16   |
|be     |13   |
|and    |12   |
|privacy|12   |
|query  |10   |
|,      |9    |
|data   |9    |
|patient|9    |
|for    |8    |
|on     |8    |
|or     |8    |
|set    |7    |
+-------+-----+
only showing top 20 rows

